# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

import warnings
warnings.filterwarnings('ignore')


You are using wmfdata v1.3.3, but v2.0.0 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2022, 11, 30)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [5]:
# Code to Suppress PySpark Warning messages
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    
    "automated_pageviews": {
        "file": "queries/automated_pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    }


# Removing since global south metrics are calculated in notebook 02b-diversity-calculation.ipynb
   # ,
   # "global_south_pageviews": {
   #     "file": "queries/global_south_pageviews.hql",
   #     "engine": "hive"
   # },
    
   # "global_south_previews": {
   #     "file": "queries/global_south_previews.hql",
   #     "engine": "hive"
   # }
    
# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
    #"mobile-heavy_pageviews": {
    #    "file": "queries/mobile_heavy_pageviews.hql",
    #    "engine": "hive" }, 
    #"mobile-heavy_previews": {
    #   "file": "queries/mobile_heavy_previews.hql",
    #   "engine": "hive" }
    
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running automated_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running page_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running unique_devices on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Combining and saving metrics

In [7]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)


In [8]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,automated_pageviews,desktop,interactions,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,
2022-02-01,9.794070e+08,6.381499e+09,1.787409e+10,9.383266e+09,1.833566e+09,1.604052e+10,1.827453e+09
2022-03-01,1.000023e+09,6.820776e+09,1.895002e+10,9.870969e+09,1.967056e+09,1.698296e+10,1.957130e+09
2022-04-01,1.257069e+09,6.517583e+09,1.793078e+10,9.357140e+09,1.773847e+09,1.615693e+10,1.878851e+09
2022-05-01,1.491689e+09,6.701587e+09,1.817494e+10,9.402734e+09,1.788005e+09,1.638694e+10,1.948874e+09
2022-06-01,1.522579e+09,6.287650e+09,1.710496e+10,8.918872e+09,1.626138e+09,1.547882e+10,2.056714e+09
2022-07-01,2.056918e+09,6.107902e+09,1.785794e+10,9.885962e+09,1.575666e+09,1.628227e+10,1.648975e+09
2022-08-01,2.403517e+09,6.176972e+09,1.889936e+10,1.079527e+10,1.634856e+09,1.726451e+10,1.665646e+09
2022-09-01,2.705377e+09,6.129059e+09,1.986889e+10,1.175190e+10,1.706285e+09,1.816260e+10,1.554616e+09
2022-10-01,2.722851e+09,6.677566e+09,2.049623e+10,1.178869e+10,1.740051e+09,1.875618e+10,1.683358e+09


In [9]:
metrics.to_csv(FILENAME, sep="\t")